In [ ]:
import pandas as pd
import numpy as np
import sklearn.feature_selection
import sklearn.metrics
import sklearn.model_selection
import sklearn.linear_model
import sklearn.covariance
import sklearn.cross_decomposition
import sklearn.feature_extraction
import sklearn.svm
import sklearn.compose
import sklearn.preprocessing
import matplotlib.pyplot as plt
import math

import statsmodels.api as sm
# from statsmodels.sandbox.regression.gmm import IV2SLS
from linearmodels.iv import IV2SLS
from numpy import dot
from numpy.linalg import norm

In [ ]:
combo_df = pd.read_csv('combined.csv')


In [ ]:
df = combo_df.__deepcopy__()

df = df[df['Price'] >= 0]
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date']).astype('int64')
df['Seasonal Price'] = df['Price']-df['Seasonal']
df.dtypes

In [ ]:
df = df[['Date', 'CSA', 'Total', '1 Unit','2 Units', '3 and 4 Units', '5 Units or More',
         'Num of Structures With 5 Units or More', 'Year', 'Month', 'filename', 'Price',
         'Year as Date', 'Population 1', 'Population 5','Population Diff', 'Population', 'Pop Growth 1 Year', 'Pop -1 Years','Pop Percent 1 Year', 'Pop Growth 5 Year', 'Pop -5 Years',
         'Pop Percent 5 Year', 'Seasonal Price']]
df

In [ ]:
print(df.Population.min())
print(df.Population.max())
print(df.Population.median())

In [ ]:
print(df.Total.min())
print(df.Total.max())
print(df.Total.median())

In [ ]:
X = df.drop(['Price'], axis = 'columns')
y = df[['Price']]

for name in df['filename'].unique():
    df[name] = np.where(df['filename']==name,1,0)
    
df['1 Unit Ratio'] = df['1 Unit']/df['Total']
df = df.dropna(axis = 'rows')




In [ ]:
# print(df.filename.head())
# print(df.head())
# print(len(df.filename.unique()))

In [ ]:
df = df.drop(['filename','Year as Date'], axis = 'columns')
df.dtypes

In [ ]:
def predict_y(df, scaler, model, target):
    X = df.drop([col for col in df.columns if 'price' in col.lower()], axis = 'columns').to_numpy()
    if scaler:
        scaler = scaler.fit(X)
        X = scaler.transform(X)
    
    y = df[target].to_numpy()
    
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y, random_state = 42)
    model = model
    model.fit(X_train, y_train)
    
    return model.predict(X_test), y_test
    


In [ ]:
y_pred_try_1, y_actual = predict_y(df, sklearn.preprocessing.StandardScaler(), sklearn.linear_model.LinearRegression(), 'Price')


In [ ]:

X1 = df.drop([col for col in df.columns if 'price' in col.lower()], axis = 'columns').to_numpy()

y1 = df['Price'].to_numpy()
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X1,y1, random_state = 42)
scaler1 = sklearn.preprocessing.StandardScaler().fit(X_train)
X_train = scaler1.transform(X_train)
X_test = scaler1.transform(X_test)
lr = sklearn.linear_model.LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)



In [ ]:
plt.plot(y_pred, color = 'blue')
plt.plot(y_pred_try_1, color = 'green')
plt.plot(y_actual, color = 'red')

In [ ]:
print(sklearn.metrics.mean_absolute_error(y_test, y_pred_try_1))
print(sklearn.metrics.mean_squared_error(y_test, y_pred_try_1))
print(sklearn.metrics.r2_score(y_test, y_pred_try_1))

In [ ]:
print(sklearn.metrics.mean_absolute_error(y_test, y_pred))
print(sklearn.metrics.mean_squared_error(y_test, y_pred))
print(sklearn.metrics.r2_score(y_test, y_pred))

In [ ]:
print(df.corr(method = 'pearson')['Price'])

In [ ]:
print(df.columns.to_list())

In [ ]:
X_ols = df.drop([col for col in df.columns if 'price' in col.lower()], axis = 'columns')

y_ols = df['Price']
X_ols_scaled = sklearn.preprocessing.StandardScaler().fit_transform(X_ols)

# x_const = sm.add_constant(X_ols_scaled)
first_model_results = sm.OLS(y_ols, X_ols_scaled).fit()
first_model_results.summary()


In [ ]:
x_const = sm.add_constant(X_ols_scaled)
second_model_results = sm.OLS(y_ols, x_const).fit()
second_model_results.summary()

In [ ]:
# import statsmodels.formula.api as smf
# # y_ols = df['Price']
# # X_ols_scaled = sklearn.preprocessing.StandardScaler().fit_transform(X_ols)
# # 'Date', 'CSA', 'Total', '1 Unit', '2 Units', '3 and 4 Units', '5 Units or More', 'Num of Structures With 5 Units or More', 'Year', 'Month', 'Price', 'Population 1', 'Population 5', 'Population Diff', 'Population', 'Pop Growth 1 Year', 'Pop -1 Years', 'Pop Percent 1 Year', 'Pop Growth 5 Year', 'Pop -5 Years', 'Pop Percent 5 Year', 'Seasonal Price', 'Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.67_sm_sa_month', 'Metro_zhvi_bdrmcnt_2_uc_sfrcondo_tier_0.33_0.67_sm_sa_month', 'Metro_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_month', 'Metro_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.67_sm_sa_month', 'Metro_zhvi_bdrmcnt_5_uc_sfrcondo_tier_0.33_0.67_sm_sa_month', 'Metro_zhvi_uc_condo_tier_0.33_0.67_sm_sa_month', 'Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month (1)', 'Metro_zhvi_uc_sfrcondo_tier_0.0_0.33_sm_sa_month', 'Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_month', 'Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month', '1 Unit Ratio'
# # x_const = sm.add_constant(X_ols_scaled)
# formula = 'Price ~ Date + CSA + Total+1 Unit+2 Units+3 and 4 Units+5 Units or More+Num of Structures With 5 Units or More+Year+Month+Population 1+Population 5+Population Diff+Population+Pop Growth 1 Year+Pop -1 Years+Pop Percent 1 Year+Pop Growth 5 Year+Pop -5 Years+Pop Percent 5 Year+1 Unit Ratio'
# third_model_results = smf.ols(formula=formula, data=df).fit()
# third_model_results.summary()

In [ ]:
# y_seas_adj = df['Seasonal Price']

# second_model_results = sm.OLS(y_seas_adj, x_const).fit()
# second_model_results.summary()

In [ ]:

# iv2sls_model = IV2SLS(y, df[endog], None, None ).fit(cov_type="unadjusted")
# print(iv2sls_model)

In [ ]:
combo_housing_income_df = pd.read_pickle('df.pickle')
combo_housing_income_df.head()

In [ ]:
combo_housing_income_df.dtypes

In [ ]:
combo_housing_income_df['Date'] = combo_housing_income_df['Date'].astype('int64')
combo_housing_income_df['Seasonal Price'] = combo_housing_income_df['Price']-combo_housing_income_df['Seasonal']

In [ ]:
combo_housing_income_df.isnull().sum()

In [ ]:
combo_housing_income_df.describe()

In [ ]:
combo_housing_income_df = combo_housing_income_df.dropna(subset=['Total', '1 Unit', 'Price'], axis = 'rows')
combo_housing_income_df.describe()

In [ ]:
combo_housing_income_df.isnull().sum()

In [ ]:
for col in combo_housing_income_df.columns:
    print(col)
    print(combo_housing_income_df[col].isnull().sum())


In [ ]:
columns_to_drop = []
for col in combo_housing_income_df.columns:
    columns_to_drop.append([col, combo_housing_income_df[col].isnull().sum()])
columns_to_drop

In [ ]:
print(combo_housing_income_df.columns.tolist())

In [ ]:
columns_to_keep = ['Date','CSA', 'Total', '1 Unit', '2 Units', '3 and 4 Units', 
                   '5 Units or More', 'Num of Structures With 5 Units or More', 'Year', 'Month', 'filename', 
                   'Price', 'Seasonal', 'Population 1', 'Population 5', 'Population Diff', 'Population', 
                   'Pop Growth 1 Year', 'Pop -1 Years', 'Pop Percent 1 Year', 'Pop Growth 5 Year', 'Pop -5 Years', 
                   'Pop Percent 5 Year', 'Income', 'Seasonal Price' ]


In [ ]:
combo_housing_income_df = combo_housing_income_df[columns_to_keep]
combo_housing_income_df.head()

In [ ]:

combo_housing_income_df.describe()

In [ ]:
print(combo_housing_income_df.dtypes)
print(combo_housing_income_df.corr(method ='spearman')['Price'])


In [ ]:
combo_housing_income_df.shape

In [ ]:
combo_housing_income_df = combo_housing_income_df.dropna(axis = 'rows')
combo_housing_income_df.shape

for name in combo_housing_income_df['filename'].unique():
#     combo_housing_income_df[name] = np.where(combo_housing_income_df['filename']==name,1,0)
    combo_housing_income_df[name] = 0
    combo_housing_income_df[name][combo_housing_income_df['filename']==name]=1
# for city in combo_housing_income_df['CSA']:
#     combo_housing_income_df[city] = 0
#     combo_housing_income_df[city][combo_housing_income_df['MSA']==city]=1
    
   
    

In [ ]:
combo_housing_income_df = combo_housing_income_df.drop('filename', axis = 'columns')
# combo_housing_income_df = combo_housing_income_df.drop('CSA', axis = 'columns') 

In [ ]:
combo_housing_income_df.dtypes

In [ ]:
y_combo_predict, y_combo_actual = predict_y(combo_housing_income_df, sklearn.preprocessing.StandardScaler(), sklearn.linear_model.LinearRegression(), 'Price')


In [ ]:
X_ols_combo = combo_housing_income_df.drop([col for col in combo_housing_income_df.columns if 'price' in col.lower()], axis = 'columns')

y_ols_combo = combo_housing_income_df['Price']
X_ols_combo_scaled = sklearn.preprocessing.StandardScaler().fit_transform(X_ols_combo)

# x_const = sm.add_constant(X_ols_scaled)
combo_ols_model_results = sm.OLS(y_ols_combo, X_ols_combo_scaled).fit()
combo_ols_model_results.summary()

In [ ]:
# y_pred_try_1
# y_actual
# y_combo_predict
# y_combo_actual



# cos_sim_orig = dot(y_pred_try_1, y_actual)/(norm(y_pred_try_1)*norm(y_actual))
# cos_sim_combo = dot(y_combo_predict, y_combo_actual)/(norm(y_combo_predict)*norm(y_combo_actual))

# print(cos_sim_orig)
# print(cos_sim_combo)

In [ ]:
print(sklearn.metrics.r2_score(y_actual,y_pred_try_1))
print(sklearn.metrics.r2_score(y_combo_actual,y_combo_predict))

In [ ]:
print(sklearn.metrics.mean_squared_error(y_actual,y_pred_try_1))
print(sklearn.metrics.mean_squared_error(y_combo_actual,y_combo_predict))

In [ ]:
print(sklearn.metrics.mean_absolute_error(y_actual,y_pred_try_1))
print(sklearn.metrics.mean_absolute_error(y_combo_actual,y_combo_predict))

In [ ]:
print(sklearn.metrics.explained_variance_score(y_actual,y_pred_try_1))
print(sklearn.metrics.explained_variance_score(y_combo_actual,y_combo_predict))

In [ ]:
print(math.dist(y_actual,y_pred_try_1))
print(math.dist(y_combo_actual,y_combo_predict))

In [ ]:
def c_v(df, scaler, model, target, cross_count, scoring):
    X = df.drop([col for col in df.columns if 'price' in col.lower()], axis = 'columns').to_numpy()
    
    if scaler:
        scaler.fit(X)
        X = scaler.transform(X)
    
    y = df[target].to_numpy()
    
    return sklearn.model_selection.cross_val_score(model, X, y, cv = cross_count, scoring = scoring).mean()
   
    

In [ ]:
def k_f(df, scaler, model, target, cross_count):
    
    kf = sklearn.model_selection.KFold(n_splits=5, random_state=1, shuffle=True)
    scores = []
    X = df.drop([col for col in df.columns if 'price' in col.lower()], axis = 'columns').to_numpy()
    
    if scaler:
        scaler.fit(X)
        X = scaler.transform(X)
    
    y = df[target].to_numpy()
    
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        X_train = X[train_index,:]
        y_train = y[train_index]
        X_test = X[test_index,:]
        y_test = y[test_index]

        model.fit(X_train, y_train)
        test_score = model.score(X_test, y_test)
        scores.append(round(test_score, 6))

    return scores

In [ ]:
original_df_cross_val = c_v(df, 
                            sklearn.preprocessing.StandardScaler(), 
                            sklearn.linear_model.LinearRegression(), 
                            'Price',
                            3, 
                            'r2')

kf_original_df_cross_val = k_f(df, 
                               sklearn.preprocessing.StandardScaler(), 
                               sklearn.linear_model.LinearRegression(), 
                               'Price',
                               5)


In [ ]:
original_df_cross_val_minmax = c_v(df, 
                                   sklearn.preprocessing.MinMaxScaler(), 
                                   sklearn.linear_model.LinearRegression(), 
                                   'Price',
                                   5, 
                                   'r2')
kf_original_df_cross_val_minmax = k_f(df, 
                                      sklearn.preprocessing.MinMaxScaler(), 
                                      sklearn.linear_model.LinearRegression(), 
                                      'Price',
                                      5)

In [ ]:
combo_df_cross_val = c_v(combo_housing_income_df, 
                         sklearn.preprocessing.StandardScaler(), 
                         sklearn.linear_model.LinearRegression(), 
                         'Price', 
                         5, 
                         'r2')

kf_combo_df_cross_val = k_f(combo_housing_income_df, 
                            sklearn.preprocessing.StandardScaler(), 
                            sklearn.linear_model.LinearRegression(), 
                            'Price', 
                            5)

In [ ]:
combo_df_cross_val_minmax = c_v(df, 
                                sklearn.preprocessing.MinMaxScaler(), 
                                sklearn.linear_model.LinearRegression(), 
                                'Price', 
                                5, 
                                'r2')

kf_combo_df_cross_val_minmax = k_f(df, 
                                   sklearn.preprocessing.MinMaxScaler(), 
                                   sklearn.linear_model.LinearRegression(), 
                                   'Price', 
                                   5)

In [ ]:
print('original df')
print(original_df_cross_val)
print('original with MinMax')
print(original_df_cross_val_minmax)
print('combo df')
print(combo_df_cross_val)
print('combo with MinMax')
print(combo_df_cross_val_minmax)

In [ ]:
print('kf original df')
print(kf_original_df_cross_val)
print('kf_original with MinMax')
print(kf_original_df_cross_val_minmax)
print('kf combo df')
print(kf_combo_df_cross_val)
print('kf combo with MinMax')
print(kf_combo_df_cross_val_minmax)

In [ ]:
sklearn.metrics.get_scorer_names()

In [ ]:
X_combo = combo_housing_income_df.drop(['Price'], axis = 'columns')

y_combo = combo_housing_income_df[['Price']]
col_transform = sklearn.compose.make_column_transformer(ohe, ['MSA','filename'])


col_transform.fit_transform(X_combo)
X_combo


In [ ]:
df = df.dropna(axis = 'rows')

# print(X1)
y1 = df1['Price'].to_numpy()
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X1,y1)
scaler1 = sklearn.preprocessing.StandardScaler(with_mean=False).fit(X_train)
X_train = scaler1.transform(X_train)
X_test = scaler1.transform(X_test)
lr = sklearn.linear_model.LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [ ]:


# pipe = sklearn.pipeline.make_pipeline(col_transform, linear_reg)

In [ ]:

X_train_combo, X_test_combo, y_train_combo, y_test_combo = sklearn.model_selection.train_test_split(X_combo,y_combo)
scaler_combo = sklearn.preprocessing.StandardScaler(with_mean=False).fit(X_train_combo)
X_train_combo = scaler_combo.transform(X_train_combo)
X_test_combo = scaler1.transform(X_test_combo)
lr_combo = sklearn.linear_model.LinearRegression()
lr.fit(X_train_combo, y_train_combo)
y_pred_combo = lr.predict(X_test_combo)

In [ ]:
# sklearn.metrics.get_scorer_names()

In [ ]:
# for scoring in sklearn.metrics.get_scorer_names():
#     print(scoring)
#     try:
#         sklearn.model_selection.cross_val_score(pipe, X, y, cv=5, scoring = scorer).mean()
#     except: continue

In [ ]:
# ind_vars = ['CSA', 'CBSA', 'Total', '1 Unit','2 Units', '3 and 4 Units', '5 Units or More',
#             'Num of Structures With 5 Units or More', 'Year', 'Month', 'Population 1', 'Population 5',
#             'Population Diff','Population', 'Pop Growth 1 Year', 'Pop -1 Years','Pop Percent 1 Year',
#             'Pop Growth 5 Year',
#             'Pop -5 Years','Pop Percent 5 Year', 'Single Unit ratio']
# dep_vars = ['Price']
#
# cols = ['CSA', 'CBSA', 'Total', '1 Unit','2 Units', '3 and 4 Units', '5 Units or More',
#         'Num of Structures With 5 Units or More', 'Year', 'Month', 'Population 1', 'Population 5',
#         'Population Diff','Population', 'Pop Growth 1 Year', 'Pop -1 Years','Pop Percent 1 Year',
#         'Pop Growth 5 Year','Pop -5 Years','Pop Percent 5 Year', 'Single Unit ratio', 'Price']
#
# # print(cols)
# no_null_df = combo_df[cols].dropna()
# # print(no_null_df.head())
# x = no_null_df[ind_vars]
# y = no_null_df[dep_vars]
# #
# print(x.head())
#
# x_const = sm.add_constant(x)
#
# first_model_results = sm.OLS(y, x_const).fit_regularized()
# first_model_results.summary()